In [ ]:
import tarfile
import zipfile
import shutil
import random
import os
import cv2
import pathlib
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import seed
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import timeit
from skimage.feature import local_binary_pattern
from PIL import Image, ImageOps

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

In [ ]:
random_seed = 42

seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy(policy)
#print('Compute dtype: %s' % policy.compute_dtype)
#print('Variable dtype: %s' % policy.variable_dtype)

# Importazione dataset

In [ ]:
%%capture
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/train.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/val.tar'
with zipfile.ZipFile("/content/gdrive/MyDrive/Progetto_VIPM/dataset/val_degraded.zip","r") as zip_ref:
    zip_ref.extractall()

In [ ]:
csv_train_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/train_clean_info.csv", dtype=str)
csv_train_file.columns = ['filename', 'label']
parent_dir = "train_set/"
labels = csv_train_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_train_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
train_set/142/train_096536.jpg
train_set/142/train_096537.jpg
train_set/142/train_096538.jpg
train_set/142/train_096539.jpg
train_set/142/train_096540.jpg
train_set/142/train_096541.jpg
train_set/142/train_096542.jpg
train_set/142/train_096543.jpg
train_set/142/train_096544.jpg
train_set/142/train_096545.jpg
train_set/142/train_096546.jpg
train_set/142/train_096547.jpg
train_set/142/train_096548.jpg
train_set/142/train_096549.jpg
train_set/142/train_096550.jpg
train_set/142/train_096551.jpg
train_set/142/train_096552.jpg
train_set/142/train_096553.jpg
train_set/142/train_096554.jpg
train_set/142/train_096555.jpg
train_set/142/train_096556.jpg
train_set/142/train_096557.jpg
train_set/142/train_096558.jpg
train_set/142/train_096559.jpg
train_set/142/train_096560.jpg
train_set/142/train_096561.jpg
train_set/142/train_096562.jpg
train_set/142/train_096563.jpg
train_set/142/train_096564.jpg
train_set/142/train_096565.jpg
train_set/142/train_

In [ ]:
csv_test_clean_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_clean_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_clean_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_clean_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set/16/val_005256.jpg
val_set/119/val_005257.jpg
val_set/68/val_005258.jpg
val_set/130/val_005259.jpg
val_set/43/val_005260.jpg
val_set/197/val_005261.jpg
val_set/101/val_005262.jpg
val_set/55/val_005263.jpg
val_set/62/val_005264.jpg
val_set/49/val_005265.jpg
val_set/17/val_005266.jpg
val_set/204/val_005267.jpg
val_set/181/val_005268.jpg
val_set/122/val_005269.jpg
val_set/170/val_005270.jpg
val_set/66/val_005271.jpg
val_set/48/val_005272.jpg
val_set/96/val_005273.jpg
val_set/86/val_005274.jpg
val_set/144/val_005275.jpg
val_set/99/val_005276.jpg
val_set/4/val_005277.jpg
val_set/30/val_005278.jpg
val_set/177/val_005279.jpg
val_set/182/val_005280.jpg
val_set/201/val_005281.jpg
val_set/168/val_005282.jpg
val_set/187/val_005283.jpg
val_set/130/val_005284.jpg
val_set/30/val_005285.jpg
val_set/145/val_005286.jpg
val_set/153/val_005287.jpg
val_set/180/val_005288.jpg
val_set/160/val_005289.jpg
val_set/37/val_005290.jpg
val_set/52/val_005291.

In [ ]:
csv_test_deg_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_deg_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_deg_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_deg_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set_degraded/16/val_005256.jpg
val_set_degraded/119/val_005257.jpg
val_set_degraded/68/val_005258.jpg
val_set_degraded/130/val_005259.jpg
val_set_degraded/43/val_005260.jpg
val_set_degraded/197/val_005261.jpg
val_set_degraded/101/val_005262.jpg
val_set_degraded/55/val_005263.jpg
val_set_degraded/62/val_005264.jpg
val_set_degraded/49/val_005265.jpg
val_set_degraded/17/val_005266.jpg
val_set_degraded/204/val_005267.jpg
val_set_degraded/181/val_005268.jpg
val_set_degraded/122/val_005269.jpg
val_set_degraded/170/val_005270.jpg
val_set_degraded/66/val_005271.jpg
val_set_degraded/48/val_005272.jpg
val_set_degraded/96/val_005273.jpg
val_set_degraded/86/val_005274.jpg
val_set_degraded/144/val_005275.jpg
val_set_degraded/99/val_005276.jpg
val_set_degraded/4/val_005277.jpg
val_set_degraded/30/val_005278.jpg
val_set_degraded/177/val_005279.jpg
val_set_degraded/182/val_005280.jpg
val_set_degraded/201/val_005281.jpg
val_set_degraded/168/val_0052

In [ ]:
train_data_dir = pathlib.Path('train_set/')
seed_train_validation = 42 # deve essere lo stesso per train_ds e val_ds
shuffle_value = True
validation_split = 0.2
batch_size = 256

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="training",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="validation",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 114153 files belonging to 251 classes.
Using 91323 files for training.
Found 114153 files belonging to 251 classes.
Using 22830 files for validation.


In [ ]:
test_clean_data_dir = pathlib.Path('val_set/')
shuffle_value = True

test_clean_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_clean_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


In [ ]:
test_deg_data_dir = pathlib.Path('val_set_degraded/')
shuffle_value = True

test_deg_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_deg_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


# Calcolo LBP

In [ ]:
# impostazioni per LBP
radius = 2
n_points = 8
method = 'ror' # rotation invariant    

In [ ]:
tr_bal_feat = []
tr_bal_labels = []

start = timeit.default_timer()
for x, y in train_ds:
  for i in range(len(x)):
    im = x[i]
    pil_im = tf.keras.preprocessing.image.array_to_img(im)
    pil_im = ImageOps.grayscale(pil_im)
    lbp_feat = local_binary_pattern(pil_im, n_points, radius, method)
    tr_bal_feat.append(lbp_feat)
    tr_bal_labels.append(y[i])
stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
tr_aug_feat = []
tr_aug_labels = []

start = timeit.default_timer()
for x, y in train_ds:
  for i in range(len(x)):
    im = x[i]
    pil_im = tf.keras.preprocessing.image.array_to_img(im)
    pil_im = ImageOps.grayscale(pil_im)
    lbp_feat = local_binary_pattern(pil_im, n_points, radius, method)
    tr_aug_feat.append(lbp_feat)
    tr_aug_labels.append(y[i])
stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
te_clean_feat = []
te_clean_labels = []

start = timeit.default_timer()
for x, y in test_clean_ds:
  for i in range(len(x)):
    im = x[i]
    pil_im = tf.keras.preprocessing.image.array_to_img(im)
    pil_im = ImageOps.grayscale(pil_im)
    lbp_feat = local_binary_pattern(pil_im, n_points, radius, method)
    te_clean_feat.append(lbp_feat)
    te_clean_labels.append(y[i])
stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  202.00621812099962


In [ ]:
te_deg_feat = []
te_deg_labels = []

start = timeit.default_timer()
for x, y in test_deg_ds:
  for i in range(len(x)):
    im = x[i]
    pil_im = tf.keras.preprocessing.image.array_to_img(im)
    pil_im = ImageOps.grayscale(pil_im)
    lbp_feat = local_binary_pattern(pil_im, n_points, radius, method)
    te_deg_feat.append(lbp_feat)
    te_deg_labels.append(y[i])
stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  202.23871862299984


# Classificatore